In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-148095", 
      subscription_id="9a7511b8-150f-4a58-8528-3e7d50216c31", 
      resource_group="aml-quickstarts-148095")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-148095
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-148095


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

my_compute_name = "mycluster"
vm_size="STANDARD_D2_V2"

if my_compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[my_compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("Compute target "+ my_compute_name + " already exists")
else:
    print("I will create a compute cluster")
    prov_config = AmlCompute.provisioning_configuration(vm_size=vm_size, min_nodes=4, max_nodes=4)

    compute_target = ComputeTarget.create(ws, my_compute_name, prov_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=120)

    print(compute_target.status.serialize())

Compute target mycluster already exists


In [35]:
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.9
- scikit-learn
- pandas
- numpy
- pip:
  - azureml-defaults

Overwriting conda_dependencies.yml


In [36]:
from azureml.core import Environment
sklearn_env = Environment.from_conda_specification(name='sklearn-env-proj', file_path='./conda_dependencies.yml')

In [49]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import ScriptRunConfig

# Specify parameter sampler
ps = RandomParameterSampling({"C": uniform(0.0, 1.0), "max_iter": choice(range(50, 100))})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=3, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = SKLearn(source_directory=".", entry_script="train.py", compute_target=compute_target)
src = ScriptRunConfig(source_directory=".",
    script="./train.py",
    compute_target=compute_target,
    environment=sklearn_env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4)

In [50]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [51]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--C', '0.7916482797373028', '--max_iter', '94']


In [52]:
print(best_run.get_file_names())
model = best_run.register_model(model_name="logreg_bank", model_path="outputs/model.joblib")

['azureml-logs/55_azureml-execution-tvmps_88f26bf6deee6bcca34e3a35cb367fb41b5b8843b64f8ff6d9c1ae4d8abe5d7d_d.txt', 'azureml-logs/65_job_prep-tvmps_88f26bf6deee6bcca34e3a35cb367fb41b5b8843b64f8ff6d9c1ae4d8abe5d7d_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_88f26bf6deee6bcca34e3a35cb367fb41b5b8843b64f8ff6d9c1ae4d8abe5d7d_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/94_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [9]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [11]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)



In [14]:
import pandas as pd
from azureml.core import Dataset

if not os.path.isdir("data"):
    os.mkdir("data")

data = pd.concat([x_train, y_train], axis=1)
data.to_csv("data/train_data.csv", index=False)

datastore = ws.get_default_datastore()
datastore.upload(src_dir="./data", target_path="trainingdata", overwrite=True, show_progress=True)
data_ds = Dataset.Tabular.from_delimited_files(path=datastore.path("trainingdata/train_data.csv"))

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [18]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    enable_early_stopping=True,
    iteration_timeout_minutes=5,
    max_concurrent_iterations=4,
    verbosity=logging.INFO,
    compute_target=compute_target,
    task='classification',
    primary_metric='accuracy',
    training_data=data_ds,
    label_column_name="y",
    n_cross_validations=3)

In [19]:
# Submit your automl run

automl_exp = Experiment(ws, "automl_exp")
automl_run = automl_exp.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on mycluster with default configuration
Running on remote compute: mycluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl_exp,AutoML_4a28d5d1-9261-43ba-b0e5-c451da973902,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

In [27]:

# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()

In [30]:
!pip install azureml-train-automl-runtime==1.30.0

In [31]:
best_run, fitted_model = automl_run.get_output()